## Problem 71 - Ordered fractions

https://projecteuler.net/problem=71

In [2]:
from math import gcd

dmax = 1_000_000
leftfrac = (0,0,0)
f37 = 3/7

for d in range(1,dmax+1):
    n = (3*d)//7
    while gcd(n,m)!=1:
        n -= 1   
    f = n/d
    if f<f37 and f>leftfrac[0]:
        leftfrac = (f,n,d)
print(leftfrac)

(0.42857128571385716, 428570, 999997)


## Problem 72 - Counting fractions

https://projecteuler.net/problem=72

In [1]:
def phi(n):
    '''Euler totient function (count coprimes of input n)'''
    phi = int(n > 1 and n)
    for p in range(2, int(n ** .5) + 1):
        if not n % p:
            phi -= phi // p
            while not n % p:
                n //= p
    if n > 1: phi -= phi // n 
    return phi

In [2]:
# number of proper fractions for denominator n would be the number of coprimes, counted by Euler totient function

def countFrac(n):
    return sum([ phi(i) for i in range(1,n+1) ])

In [3]:
countFrac(8)

21

In [6]:
import time

tic = time.perf_counter()

res = countFrac(1_000_000)
print("Problem 72 result =",res)

toc = time.perf_counter()
print(f"Solved in {toc - tic:0.4f} seconds")

Problem 72 result = 303963552391
Solved in 22.7452 seconds


## Problem 73 - Counting fractions in a range

https://projecteuler.net/problem=73

In [67]:
from math import gcd

fmin = 1/3
fmax = 1/2

#dmax = 8
dmax = 12_000

count = []
for d in range(1,dmax+1):
    for n in range(d//3,d//2+1):
        if gcd(n,d)==1:
            f = n/d
            if f>fmin and f<fmax:
                count.append((f,n,d))

print(len(count))

7295372


## Problem 74 - Digit factorial chains

https://projecteuler.net/problem=74

In [56]:
from math import factorial
from collections import defaultdict

# Only need the factorials of digits 0-9, caching them to speed up computation
factCache = {}
for i in range(10):
    factCache[i] = factorial(i)

# Many chains have repeating tails, caching them to speed up computation
chains = defaultdict(list)
    
def factorialChain(nstart,lmax=0):
    chain = [nstart]
    n = nstart
    while True:
        n = sum([ factCache[i] for i in [ int(s) for s in str(n)] ])
        if n in chains.keys(): # chains for current value already completed!
            chain += chains[n]
            chains[nstart] = chain
            return chain
        if n not in chain:
            chain.append(n)
        else:
            break
        if lmax and len(chain)>lmax:
            print("Longer than {}, skipping".format(lmax)) # never really happens...
            return []
    chains[nstart] = chain
    return chain

factorialChain(169)

[169, 363601, 1454]

In [57]:
c = 0
for n in range(1,1_000_000):
    chain = factorialChain(n,lmax=60)
    if len(chain)==60:
        c+=1
print("Solution 72 =",c)

Solution 72 = 402


## Problem 75 - Singular integer right triangles

https://projecteuler.net/problem=75

In [4]:
def triple(m,n):
    a = m**2-n**2
    b = 2*m*n
    c = m**2+n**2
    return a,b,c

from collections import defaultdict

def wireLenghts(Lmax = 1000):
    W = defaultdict(list)
    n = 1
    # generate all primitive pythagorean triples from Euclid formula
    while True:
        m = n+1
        while True:
            t = triple(m,n)
            L = sum(t)
            if L>Lmax:
                break
            # generate also non primitive triples in case they can lead to a valid wire lenght
            k=1
            while True:
                tk = [k*x for x in t]
                L = sum(tk)
                if L>Lmax:
                    break
                # check for duplicates (different order in the triple)
                tks = sorted(tk)
                if tks not in W[L]:
                    W[L].append(tks)
                k+=1
            m+=1
        if m==n+1:
            break
        n+=1
    return W

In [5]:
W = wireLenghts(200)
W[120]

[[30, 40, 50], [24, 45, 51], [20, 48, 52]]

In [8]:
W = wireLenghts(1_500_000)
print(sum([1 for L in W.keys() if len(W[L])==1 ]))

161667


## Problem 76 - Counting summations

https://projecteuler.net/problem=76

In [3]:
# esplicitly generating partitions, even if I already know it won't scale well, even with caching...

cacheSums = {}

def findSums(N):
    if N==1:
        return [[1]]
    if N in cacheSums.keys():
        return cacheSums[N]
    sums = [[N]]
    for n in range(N-1,0,-1):
        for s in findSums(N-n):
            si = sorted([n]+s,reverse=True)
            if si not in sums:
                sums.append(si)
    cacheSums[N] = sums
    return sums

findSums(5)

[[5], [4, 1], [3, 2], [3, 1, 1], [2, 2, 1], [2, 1, 1, 1], [1, 1, 1, 1, 1]]

In [124]:
# Borrowing solution of Problem 31 to only count paritions,  adding caching to speed up calculation

def countCoins(S, m, n, cache={}): 
    '''
    Returns count of ways we can sum S[0...m-1] coins to get sum n 
    S = list of coin values
    m = how many coin values to use
    n = total value to obtain
    '''
    
    # If n is 0 then there is 1 solution (do not include any coin) 
    if n == 0: 
        return 1
  
    # If n is less than 0 then no solution exists 
    if n < 0: 
        return 0
    
    # If there are no coins available anymore and n is still greater than 0, then no solution exists
    if m<=0 and n>=0: 
        return 0
  
    # Recursive solution:
    # Sum of solutions including S[m-1] and solutions excluding S[m-1]
    if (m,n) in cache.keys():
        return cache[(m,n)]
    count = countCoins(S, m-1, n, cache) + countCoins(S, m, n-S[m-1], cache)
    cache[(m,n)] = count
    return count

def countSums(N):
    values = [i for i in range(1,N)]
    cache = {} # initialize cache
    return countCoins(values, len(values), N, cache)

In [125]:
countSums(5)

6

In [126]:
countSums(100)

190569291

## Problem 77 - Prime summations

https://projecteuler.net/problem=77

In [127]:
from ProjectEuler import generatePrimes

# Generate a large number of primes to avoid recomputing them for each number
nmax = 100000
primes = []
for p in generatePrimes():
    if p>=nmax:
        break
    primes.append(p)
    
def countSumsPrimes(N):
    cache = {} # initialize cache
    # select primes list smaller than N
    i = 0
    for p in primes:
        if p>N:
            break
        i+=1
    primesN = primes[:i]
    return countCoins(primesN, len(primesN), N, cache)

i = 1
while True:
    c = countSumsPrimes(i)
    if c>5000:
        print(i,c)
        break
    i+=1

71 5007


## Problem 78 - Coin partitions

https://projecteuler.net/problem=78

https://en.wikipedia.org/wiki/Partition_function_(number_theory)#Generating_function

https://mathworld.wolfram.com/PartitionFunctionP.html

$P(n)=\sum_{k=1}^n (-1)^{(k+1)} \left[ P(n-1/2k(3k-1)) + P(n-1/2k(3k+1)) \right]$

In [221]:
def P(n,cache={}):
    if n<0: # negative argument has 0 partitions
        return 0
    if n==0 or n==1:
        return 1
    if n in cache.keys():
        return cache[n]
    p = 0
    for k in range(1,n+1):
        #p += (-1)**(k+1) * ( P(n-k*(3*k-1)//2) + P(n-k*(3*k+1)//2) )
        a_k = n-k*(3*k-1)//2
        b_k = n-k*(3*k+1)//2
        #print(a_k,b_k) # as soon as it reaches negative arguments, all subsequent k-th arguments are negative!
        if a_k<0 and b_k<0: # no need to compute terms for negative arguments
            break
        p += pow(-1,(k+1)%2) * ( P(a_k , cache ) + P(b_k , cache ) )
    cache[n]=p
    return p

In [222]:
cache = {}
for n in range(1,11):
    print(n,P(n,cache))

1 1
2 2
3 3
4 5
5 7
6 11
7 15
8 22
9 30
10 42


In [227]:
cache = {}
n = 1
while True:
    if P(n,cache)%1_000_000==0:
        print(n)
        break
    n+=1

55374


## Problem 79 - Passcode derivation

https://projecteuler.net/problem=79

In [36]:
with open("data/p079_keylog.txt") as f:
    values = [ int(l) for l in f.readlines() ]

from collections import defaultdict

# database of possible digit order, to be navigated as a graph with a BFS-like algorithm
order = defaultdict(list)
for v in values:
    n = [ int(a) for a in str(v) ]
    if n[1] not in order[ n[0] ]:
        order[ n[0] ].append(n[1])
    if n[2] not in order[ n[0] ]:      
        order[ n[0] ].append(n[2])
    if n[2] not in order[ n[1] ]:    
        order[ n[1] ].append(n[2])

# initialize passdoces with all possible digits
passcodes = []
for d in order.keys():
    passcodes.append([d])

# BFS to build all possible passcodes
while True:
    if len(passcodes)==1:
        break
    pc = passcodes.pop(0)
    for nextdig in order[pc[-1]]:
        # The problem is asking for shortest passcode, I therefire assume 
        # there will be no repetition and all digits will be used once: 
        # removing passcodes (graph paths) with repetition
        if pcnew not in pc: 
            pcnew = pc+[nextdig]
            passcodes.append(pcnew)
            
print(int("".join([ str(p) for p in passcodes[0]])))

73162890


## Problem 80 - Square root digital expansion

https://projecteuler.net/problem=80

For the first one hundred natural numbers, find the total of the digital sums of the first one hundred decimal digits for all the irrational square roots

In [66]:
from decimal import *
from math import sqrt

# using decimal to compute arbitrary precision sqrt, exceeding the needed precision to avoid rounding errors
getcontext().prec = 105 

s = 0
for n in range(1,101):
    if sqrt(n)==int(sqrt(n)): # skip perfect squares
        continue    
    s += sum([ int(d) for d in str(Decimal(n).sqrt())[:101] if d!="." ])
print(s)

40886
